# アルゴリズムのパイプラインを改造するためのテンプレート


## はじめに
このNotebookは、04-02.で試してみたパイプラインを改造し、Part3.で学習したAlphaLensを使ってアルファのチューニングを行うためのテンプレートです

### ちょっとしたTips
アルゴリズムのパイプラインを改造する場合、パイプラインのカラムには、アルファのみ入れるようにクセをつけておくと、get_clean_factor_and_forward_returnsの引数処理が汎用的になります。


## 下準備１：（必須）必要なパッケージのインポート
アルファ（ファクター）の構築に必要なパッケージと、Alphalensの利用に必要なパッケージのインポートを行います。
Alphalens 関連のパッケージ以外は、ここまでのNotebookで扱ってきたものです。
必要に応じてパッケージを追加してください。


In [ ]:
# Notebook上でパイプラインを実行する上で必要なパッケージ
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

# ファクター 
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.data import Fundamentals

# ユニバース
from quantopian.pipeline.filters import QTradableStocksUS

# Alphalens
from alphalens.tears import create_information_tear_sheet
from alphalens.utils import get_clean_factor_and_forward_returns


## 下準備２：（オプション）変数などの指定（オプション）

make_pipeline()の外側で指定している変数を参照しているような場合には、以下のブロックで指定します。


In [ ]:
# Alphalensの分析期間をデフォルト（1D,5D,10D）から変えたいときに利用
periods = (1, 5, 20)

# 変数の指定があれば、ここで便宜的にセットしてしまう


## メイン作業：make_pipline() を貼り付け

make_pipeline()のロジックをコピペします。**以上で作業は終了です**。


In [ ]:
from quantopian.pipeline.data import factset
 
OUTLIER_THRESHOLD = 3 # Maximum zscore that is not an outlier
ZERO_THRESHOLD = 0.1 # Minimum zscore we allow before dropping security
 
    
def make_pipeline():
    # Setting up the variables
    # marketcap に対して売上が比較的小さい会社を買う（過去一年）
    
    revenue_ltm = factset.Fundamentals.sales_ltm.latest/\
                  factset.Fundamentals.mkt_val_public.latest
    fcf = factset.Fundamentals.free_cf_fcfe_qf.latest / \
                   factset.Fundamentals.entrpr_val_qf.latest
    
    alpha_factor = revenue_ltm.log() + fcf.log()
    
    # Standardized logic for each input factor after this point
    alpha_w = alpha_factor.winsorize(
        min_percentile=0.10,
        max_percentile=0.98,
        mask=QTradableStocksUS() & (revenue_ltm > 0) & alpha_factor.isfinite()
    )
    alpha_z = alpha_w.zscore()
    
    outlier_filter = alpha_z.abs() < OUTLIER_THRESHOLD
    non_zero_filter = alpha_z.abs() > ZERO_THRESHOLD
    
    universe = QTradableStocksUS() & \
               outlier_filter & \
               non_zero_filter
 
    pipe = Pipeline(
        columns={
            'alpha_z': alpha_z,
        },
        screen=universe
    )
    return pipe

# アルファの予測精度分析
コード変更不要

In [ ]:
# Alphalens で分析させる期間（直近から２年分を指定）
import datetime
start_date = "2016-1-1"#(datetime.datetime.now() - datetime.timedelta(days=250)).strftime('%Y-%m-%d')
end_date   = "2018-1-1"#(datetime.datetime.now() - datetime.timedelta(days=3)).strftime('%Y-%m-%d')

# パイプライン実行
pipeline_output = run_pipeline(make_pipeline(), start_date, end_date)

# Alphalenseで必要なデータの準備と実行（この３行を変える必要は多分殆どない）
pricing_data = get_pricing(pipeline_output.index.levels[1], start_date, end_date, fields='open_price')
factor_data = get_clean_factor_and_forward_returns(factor=pipeline_output,
                                                            prices=pricing_data,
                                                            periods=periods)
create_information_tear_sheet(factor_data)

# リターン分析
コード変更不要


In [ ]:
from alphalens.tears import create_returns_tear_sheet
create_returns_tear_sheet(factor_data)

## 最後に
納得のいくパイプラインが出来上がったら、Algorithmの方に張り付けてバックテストを実行する。
